In [15]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

In [16]:
train_df = pd.read_csv('/Users/sola/Downloads/open/train.csv')
test_df = pd.read_csv('/Users/sola/Downloads/open/test.csv')

In [17]:
# 결측치에 평균 넣어주기
train_df=train_df.fillna(train_df.mean())
test_df=test_df.fillna(test_df.mean())

/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_3597/1987006743.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df=train_df.fillna(train_df.mean())
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_3597/1987006743.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df=test_df.fillna(test_df.mean())


In [18]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [19]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [20]:
# 범주형을 수치형으로 변환하는 과정
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [21]:
hgb = HistGradientBoostingClassifier(random_state=37)

# 교차검증
scores = cross_validate(hgb, train_x, train_y, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.6188375350140056


In [22]:
from sklearn.inspection import permutation_importance

# 학습시키기
hgb.fit(train_x, train_y)
#특성중요도 확인
# result = permutation_importance(hgb, train_x, train_y, n_repeats=10,random_state=37, n_jobs=-1)
# print(result.importances_mean)

HistGradientBoostingClassifier(random_state=37)

In [23]:
# hgb.score(test_input, test_target)

In [24]:
preds = hgb.predict(test_x)

In [25]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [26]:
submit = pd.read_csv('/Users/sola/Downloads/open/sample_submission.csv')

In [27]:
submit['Y_Class'] = preds

In [28]:
submit.to_csv('/Users/sola/Downloads/open/baseline_submission4.csv', index=False)

히스토그램 기반 그레이디언트 부스팅 : submission3
-> 결측치에 전부 0 입력
정확도 0.541749409

히스토그램 기반 그레이디언트 부스팅 : submission4
-> 결측치에 컬럼별 평균치넣고 아무것도 없는 컬럼에는 0 입력
정확도